In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from spatial import get_force_boundary, map_to_spatial_unit
from utils import Month, MonthRange, extract_crime_data, extract_monthly_crime_data

In [3]:
FORCE = "West Yorkshire"
filter = {"Crime type": "Anti-social behaviour"}
boundary = get_force_boundary(FORCE)

In [4]:
latest = extract_crime_data(FORCE, filters=filter)

In [5]:
chunks = []
for month in MonthRange(Month(2010, 12), end=Month(2025, 5)):
    chunks.append(extract_monthly_crime_data(FORCE, month, filters=filter))

all_crimes = pd.concat([*chunks, latest])
all_crimes

,Month,Reported by,Falls within,Location,LSOA code,LSOA name,Crime type,geometry
Crime ID,,,,,,,,
NaN,2010-12,West Yorkshire Police,West Yorkshire Police,On or near Old Lane,E01010646,Bradford 001A,Anti-social behaviour,POINT (409093.994 449079.032)
NaN,2010-12,West Yorkshire Police,West Yorkshire Police,On or near Wharfe Park,E01010646,Bradford 001A,Anti-social behaviour,POINT (408270.001 449805.954)
NaN,2010-12,West Yorkshire Police,West Yorkshire Police,On or near Main Street,E01010648,Bradford 001C,Anti-social behaviour,POINT (407467.983 449830.986)
NaN,2010-12,West Yorkshire Police,West Yorkshire Police,On or near Crossbeck Road,E01010692,Bradford 001D,Anti-social behaviour,POINT (411995.974 447223.996)
NaN,2010-12,West Yorkshire Police,West Yorkshire Police,On or near Church Street,E01010695,Bradford 002D,Anti-social behaviour,POINT (411609.02 447806.042)
...,...,...,...,...,...,...,...,...
NaN,2025-04,West Yorkshire Police,West Yorkshire Police,On or near John Street,E01011863,Wakefield 044D,Anti-social behaviour,POINT (446750.007 410878.044)
NaN,2025-04,West Yorkshire Police,West Yorkshire Police,On or near Beech Street,E01011863,Wakefield 044D,Anti-social behaviour,POINT (446414.024 410431.008)
NaN,2025-04,West Yorkshire Police,West Yorkshire Police,On or near Princess Avenue,E01011863,Wakefield 044D,Anti-social behaviour,POINT (446666.975 410800.005)


In [31]:
timespan = len(all_crimes.Month.unique()) / 12

mapped_crimes, features = map_to_spatial_unit(all_crimes, boundary, "MSOA")
agg_crimes = (
    mapped_crimes.groupby("spatial_unit")["Crime type"]
    .count()
    # .unstack(level="Month", fill_value=0)
    .sort_index()
).rename("total")

In [28]:
g = features.join(agg_crimes).fillna(0)
# g.explore("total")
g["rate"] = g["total"] / g.area / timespan * 1_000_000

/tmp/ipykernel_156735/958010282.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  g = features.join(agg_crimes).fillna(0)


In [30]:
g.explore("rate")